# 用户价值预测
* 根据各个参数预测用户的价值

[kggle链接](https://www.kaggle.com/c/santander-value-prediction-challenge)

In [1]:
# 导入库
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, model_selection, metrics
import lightgbm as lgb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
DATAPATH="D:/dataset/SVPC/"
train_df = pd.read_csv(DATAPATH+"train.csv")
test_df = pd.read_csv(DATAPATH+"test.csv")
print("Train rows and columns : ", train_df.shape)
print("Test rows and columns : ", test_df.shape)

Train rows and columns :  (4459, 4993)
Test rows and columns :  (49342, 4992)


In [4]:
unique_df = train_df.nunique().reset_index()
unique_df.columns = ["col_name", "unique_count"]
constant_df = unique_df[unique_df["unique_count"]==1]
constant_df.shape

(256, 2)

In [8]:
### Get the X and y variables for building model ###
train_X = np.log1p(train_df.drop(constant_df.col_name.tolist() + ["ID"], axis=1))

In [37]:
LABEL_COLUMN = 'target'
FEATURE_COLUMNS = [f for f in train_X if not f == LABEL_COLUMN]

In [38]:
FEATURE_COLUMNS

['48df886f9',
 '0deb4b6a8',
 '34b15f335',
 'a8cb14b00',
 '2f0771a37',
 '30347e683',
 'd08d1fbe3',
 '6ee66e115',
 '20aa07010',
 'dc5a8f1d8',
 '11d86fa6a',
 '77c9823f2',
 '8d6c2a0b2',
 '4681de4fd',
 'adf119b9a',
 'cff75dd09',
 '96f83a237',
 'b8a716ebf',
 '6c7a4567c',
 '4fcfd2b4d',
 'f3b9c0b95',
 '71cebf11c',
 'd966ac62c',
 '68b647452',
 'c88d108c9',
 'ff7b471cd',
 '0d866c3d7',
 'bc3f77679',
 'bd8f989f1',
 '0eff5bf95',
 '22ed6dba3',
 '92b13ebba',
 '233c7c17c',
 '2cb4d123e',
 '87ffda550',
 '822e49b95',
 '316b978cd',
 'd04e16aed',
 '5d5c5ce6d',
 'ec863cb52',
 '11ad148bd',
 'ea18d720e',
 '408d86ce9',
 '69d8b4020',
 'b0868a049',
 '80b14398e',
 'ecdef52b2',
 'c36bcacc5',
 '151d318cd',
 'ab0cddb90',
 '2d6bd8275',
 'dfdf4b580',
 'c87722678',
 'a8c320153',
 'a04684f1f',
 'fa977f17b',
 'feed9d437',
 '645b47cde',
 '7298ca1ef',
 'd80abf8bc',
 '654dd8a3b',
 '7fc39583c',
 '8c94b6675',
 'e421c414e',
 '964335fdf',
 '5a86cabd0',
 '6cb207ac9',
 'c1bc828da',
 'e90ed19da',
 'c4d38135f',
 '968b01ca3',
 '26b4

In [18]:
import tensorflow as tf
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURE_COLUMNS]

In [12]:
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols, 
                                          hidden_units=[64, 128], 
                                          model_dir='./models/dnnregressor')

Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001BF4DBB9CC0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_

In [45]:
def input_fn(data_set):
    feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURE_COLUMNS}
    labels = tf.constant(data_set[[LABEL_COLUMN]].values)
    return feature_cols, labels

In [46]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings("ignore")
regressor.fit(input_fn=lambda:  input_fn(train_X), steps=500)

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/dnnregressor\model.ckpt.
INFO:tensorflow:loss = 287.117, step = 0
INFO:tensorflow:global_step/sec: 1.70438
INFO:tensorflow:loss = 11.882409, step = 100 (58.705 sec)
INFO:tensorflow:global_step/sec: 10.4219
INFO:tensorflow:loss = 3.2386947, step = 200 (9.563 sec)
INFO:tensorflow:global_step/sec: 10.5515
INFO:tensorflow:loss = 2.8546598, step = 300 (9.478 sec)
INFO:tensorflow:global_step/sec: 10.5516
INFO:tensorflow:loss = 2.77593, step = 400 (9.477 sec)
INFO:tensorflow:Saving checkpoints for 500 into ./models/dnnregressor\model.ckpt.
INFO:tensorflow:Loss for final step: 2.7308235.


DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x000001BF4A910B38>, 'hidden_units': [64, 128], 'feature_columns': (_RealValuedColumn(column_name='48df886f9', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='0deb4b6a8', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='34b15f335', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='a8cb14b00', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='2f0771a37', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='30347e683', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='d08d1fbe3', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='6ee66e1

In [47]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
ev = regressor.evaluate(input_fn=lambda:  input_fn(train_X), steps=1)
print('ev: {}'.format(ev))

INFO:tensorflow:Starting evaluation at 2018-07-17-11:38:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/dnnregressor\model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-07-17-11:39:39
INFO:tensorflow:Saving dict for global step 500: global_step = 500, loss = 2.7304516
ev: {'loss': 2.7304516, 'global_step': 500}


In [48]:
test_X = np.log1p(test_df.drop(constant_df.col_name.tolist() + ["ID"], axis=1))

In [ ]:
def tinput_fn(data_set):
    feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURE_COLUMNS}
    return feature_cols
predict = regressor.predict(input_fn=lambda:  tinput_fn(test_X), as_iterable=False)

In [9]:
np.save(DATAPATH+"Xlog1p.h5",X)
np.save(DATAPATH+"Ylog1p.h5",Y)
np.save(DATAPATH+"Tlog1p.h5",T)

In [3]:
# 导入库
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, model_selection, metrics
import lightgbm as lgb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
DATAPATH="D:/dataset/SVPC/"

X=np.load(DATAPATH+"Xlog1p.h5.npy")
Y=np.load(DATAPATH+"Ylog1p.h5.npy")
T=np.load(DATAPATH+"Tlog1p.h5.npy")

In [4]:
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURE_COLUMNS]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])